#📌 Extracción

In [22]:
# ===========================================
# EXTRACCIÓN "PEGA-Y-CORRE" PARA GOOGLE COLAB
# Soporta: JSON, JSONL, CSV, Excel, Parquet
# Desde: Google Drive o GitHub (raw)
# ===========================================
import pandas as pd, numpy as np, json, io, re, os
from pathlib import Path

# ========= 1) CONFIGURA LA FUENTE =========
SOURCE_MODE = "github"   # "github" o "drive"

# A) Si usas GITHUB (RAW):
GITHUB_RAW_URL = "https://raw.githubusercontent.com/fjvalencia7/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# ========= 2) HELPERS DE LECTURA ROBUSTA =========
def _read_json_like_text(txt: str) -> pd.DataFrame:
    txt = txt.strip()
    # JSON Lines (una fila JSON por línea)
    if "\n" in txt and txt.lstrip().startswith("{"):
        return pd.read_json(io.StringIO(txt), lines=True)
    # JSON normal (lista o dict)
    obj = json.loads(txt)
    return pd.json_normalize(obj)

def read_from_url(url: str) -> pd.DataFrame:
    import requests
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    # Detecta por extensión
    u = url.lower()
    if u.endswith((".csv",)):
        return pd.read_csv(io.StringIO(r.text))
    if u.endswith((".xlsx", ".xls")):
        return pd.read_excel(io.BytesIO(r.content))
    if u.endswith((".parquet",)):
        return pd.read_parquet(io.BytesIO(r.content))
    # JSON (por contenido)
    return _read_json_like_text(r.text)

def read_from_path(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"No existe el archivo: {path}")
    low = str(p).lower()
    if low.endswith(".csv"):
        return pd.read_csv(p)
    if low.endswith((".xlsx", ".xls")):
        return pd.read_excel(p)
    if low.endswith(".parquet"):
        return pd.read_parquet(p)
    # JSON / JSONL
    try:
        # JSON Lines
        return pd.read_json(p, lines=True)
    except Exception:
        # JSON normal
        with open(p, "r", encoding="utf-8") as f:
            txt = f.read()
        return _read_json_like_text(txt)

# ========= 3) EXTRACCIÓN =========
if SOURCE_MODE == "github":
    df_raw = read_from_url(GITHUB_RAW_URL)
elif SOURCE_MODE == "drive":
    from google.colab import drive
    drive.mount('/content/drive')
    df_raw = read_from_path(DRIVE_PATH)
else:
    raise ValueError("SOURCE_MODE debe ser 'github' o 'drive'.")

print("✅ Extracción completa.")
print("Dimensiones RAW:", df_raw.shape)
display(df_raw.head())

# ========= 4) LIMPIEZA BÁSICA DE TIPOS =========
df = df_raw.copy()

# Conversión de numéricos típicos (Telco churn)
for col in ["tenure", "MonthlyCharges", "TotalCharges"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Mapeo genérico a booleanos para columnas tipo Sí/No/True/False
def to_bool_series(s):
    mapping = {
        "yes": True, "no": False, "si": True, "sí": True,
        "true": True, "false": False, "y": True, "n": False,
        "1": True, "0": False
    }
    return s.astype(str).str.strip().str.lower().map(mapping)

bool_candidates = [
    "Partner","Dependents","PhoneService","PaperlessBilling",
    "MultipleLines","OnlineSecurity","OnlineBackup",
    "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Churn"
]
for col in bool_candidates:
    if col in df.columns:
        df[col] = to_bool_series(df[col])

# Contract como ordinal (si existe)
if "Contract" in df.columns:
    order = ["Month-to-month","One year","Two year"]
    df["Contract"] = pd.Categorical(df["Contract"], categories=order, ordered=True)

print("\n✅ Tipos ajustados.")
print("Dimensiones LIMPIO:", df.shape)
display(df.head())
print("\nDtypes:")
print(df.dtypes)

# ========= 5) CHEQUEOS RÁPIDOS DE CALIDAD =========
print("\nNulos por columna (Top 10):")
display(df.isna().sum().sort_values(ascending=False).head(10))

# ========= 6) EXPORTACIÓN OPCIONAL =========
OUT_CSV = "TelecomX_Data_clean.csv"
df.to_csv(OUT_CSV, index=False)
print(f"\n💾 CSV limpio guardado en: {OUT_CSV}")

# ========= 7) MINI DICCIONARIO DE DATOS =========
def quick_dictionary(dataframe):
    rows = []
    for c in dataframe.columns:
        s = dataframe[c]
        rows.append({
            "columna": c,
            "tipo": str(s.dtype),
            "nulos": int(s.isna().sum()),
            "cardinalidad": int(s.nunique()),
            "ejemplos": [str(v) for v in s.dropna().unique()[:5]]
        })
    return pd.DataFrame(rows).sort_values("columna")

print("\n📘 Diccionario de datos (vista rápida):")
display(quick_dictionary(df))

✅ Extracción completa.
Dimensiones RAW: (7267, 21)


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4



✅ Tipos ajustados.
Dimensiones LIMPIO: (7267, 21)


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,False,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,False,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,True,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,True,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,True,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4



Dtypes:
customerID                    object
Churn                         object
customer.gender               object
customer.SeniorCitizen         int64
customer.Partner              object
customer.Dependents           object
customer.tenure                int64
phone.PhoneService            object
phone.MultipleLines           object
internet.InternetService      object
internet.OnlineSecurity       object
internet.OnlineBackup         object
internet.DeviceProtection     object
internet.TechSupport          object
internet.StreamingTV          object
internet.StreamingMovies      object
account.Contract              object
account.PaperlessBilling      object
account.PaymentMethod         object
account.Charges.Monthly      float64
account.Charges.Total         object
dtype: object

Nulos por columna (Top 10):


,0
Churn,224
customerID,0
customer.gender,0
customer.SeniorCitizen,0
customer.Partner,0
customer.Dependents,0
customer.tenure,0
phone.PhoneService,0
phone.MultipleLines,0
internet.InternetService,0



💾 CSV limpio guardado en: TelecomX_Data_clean.csv

📘 Diccionario de datos (vista rápida):


,columna,tipo,nulos,cardinalidad,ejemplos
1,Churn,object,224,2,"[False, True]"
19,account.Charges.Monthly,float64,0,1585,"[65.6, 59.9, 73.9, 98.0, 83.9]"
20,account.Charges.Total,object,0,6531,"[593.3, 542.4, 280.85, 1237.85, 267.4]"
16,account.Contract,object,0,3,"[One year, Month-to-month, Two year]"
17,account.PaperlessBilling,object,0,2,"[Yes, No]"
18,account.PaymentMethod,object,0,4,"[Mailed check, Electronic check, Credit card (..."
5,customer.Dependents,object,0,2,"[Yes, No]"
4,customer.Partner,object,0,2,"[Yes, No]"
3,customer.SeniorCitizen,int64,0,2,"[0, 1]"
2,customer.gender,object,0,2,"[Female, Male]"


#🔧 Transformación

In [23]:
# ===========================================
# TRANSFORMACIÓN TELECOMX (pega-y-corre)
# Limpieza + Imputación + Feature Eng + One-hot
# ===========================================
import pandas as pd
import numpy as np

# 0) Cargar df si no existe (desde extracción previa)
if "df" not in globals():
    try:
        df = pd.read_csv("TelecomX_Data_clean.csv")
        print("Cargado: TelecomX_Data_clean.csv")
    except Exception as e:
        raise RuntimeError("No encontré 'df' en memoria ni 'TelecomX_Data_clean.csv'. Ejecuta primero la extracción.") from e

df_tr = df.copy()

# 1) Limpieza general de texto en columnas tipo objeto/categórica
for c in df_tr.columns:
    if df_tr[c].dtype == "object":
        df_tr[c] = df_tr[c].astype(str).str.strip()

# 2) Asegurar tipos típicos (por si llegaron como texto)
num_cols_expect = [c for c in ["tenure","MonthlyCharges","TotalCharges"] if c in df_tr.columns]
for c in num_cols_expect:
    df_tr[c] = pd.to_numeric(df_tr[c], errors="coerce")

# 3) Variables booleanas estándar (si existen) -> map a 0/1 más adelante
bool_cols = [c for c in [
    "Partner","Dependents","PhoneService","PaperlessBilling","MultipleLines",
    "OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",
    "StreamingTV","StreamingMovies","Churn"
] if c in df_tr.columns]

# 4) Imputación de nulos:
#    - Numéricos: mediana
#    - Categóricos: moda
num_cols = [c for c in df_tr.columns if pd.api.types.is_numeric_dtype(df_tr[c])]
cat_cols = [c for c in df_tr.columns if df_tr[c].dtype == "object" or str(df_tr[c].dtype) == "category"]

for c in num_cols:
    med = df_tr[c].median()
    df_tr[c] = df_tr[c].fillna(med)

for c in cat_cols:
    if df_tr[c].isna().any():
        moda = df_tr[c].mode(dropna=True)
        if len(moda) > 0:
            df_tr[c] = df_tr[c].fillna(moda.iloc[0])
        else:
            df_tr[c] = df_tr[c].fillna("Desconocido")

# 5) Feature engineering útil para churn/retail telco
#    5.1 Tenure en años y buckets
if "tenure" in df_tr.columns:
    df_tr["tenure_years"] = (df_tr["tenure"] / 12).round(2)
    df_tr["tenure_bucket"] = pd.cut(
        df_tr["tenure"],
        bins=[-1, 6, 12, 24, 36, 60, np.inf],
        labels=["0-6m","7-12m","13-24m","25-36m","37-60m","60m+"]
    )

#    5.2 Número total de servicios contratados (suma de booleanos)
svc_flags = [c for c in [
    "PhoneService","MultipleLines","OnlineSecurity","OnlineBackup",
    "DeviceProtection","TechSupport","StreamingTV","StreamingMovies"
] if c in df_tr.columns]
def to01(s):
    # convierte True/False/Yes/No/Si/Sí/0/1 a 0/1
    mapping = {"yes":1,"si":1,"sí":1,"true":1,"y":1,"1":1,
               "no":0,"false":0,"n":0,"0":0}
    return (s.astype(str).str.strip().str.lower().map(mapping)).astype("float").fillna(0)

for c in svc_flags:
    if df_tr[c].dtype != bool and not pd.api.types.is_numeric_dtype(df_tr[c]):
        df_tr[c] = to01(df_tr[c]).astype(int)
    elif df_tr[c].dtype == bool:
        df_tr[c] = df_tr[c].astype(int)

df_tr["total_services"] = df_tr[svc_flags].sum(axis=1) if svc_flags else 0

#    5.3 Flags de contrato e internet (si existen)
if "Contract" in df_tr.columns:
    df_tr["contract_monthly"] = (df_tr["Contract"].astype(str).str.contains("Month", case=False)).astype(int)
    df_tr["contract_1yr"]     = (df_tr["Contract"].astype(str).str.contains("One", case=False)).astype(int)
    df_tr["contract_2yr"]     = (df_tr["Contract"].astype(str).str.contains("Two", case=False)).astype(int)

if "InternetService" in df_tr.columns:
    df_tr["internet_fiber"] = (df_tr["InternetService"].astype(str).str.contains("Fiber", case=False)).astype(int)
    df_tr["internet_dsl"]   = (df_tr["InternetService"].astype(str).str.contains("DSL", case=False)).astype(int)

#    5.4 Relación cargos: mensual vs total (si ambas existen y total>0)
if all(c in df_tr.columns for c in ["MonthlyCharges","TotalCharges"]):
    df_tr["monthly_to_total_ratio"] = np.where(df_tr["TotalCharges"]>0,
                                               df_tr["MonthlyCharges"]/df_tr["TotalCharges"],
                                               0)

# 6) Target y columnas ID
target_col = "Churn" if "Churn" in df_tr.columns else None
id_cols = [c for c in ["customerID","CustomerID","id"] if c in df_tr.columns]

# 7) Codificación:
#    - Booleanos en 0/1
for c in bool_cols:
    if c in df_tr.columns and df_tr[c].dtype == bool:
        df_tr[c] = df_tr[c].astype(int)
    elif c in df_tr.columns and not pd.api.types.is_numeric_dtype(df_tr[c]):
        df_tr[c] = to01(df_tr[c]).astype(int)

#    - One-hot para categóricas (excluye ID y target)
ohe_drop = id_cols + ([target_col] if target_col else [])
cat_for_ohe = []
for c in df_tr.columns:
    if c in ohe_drop:
        continue
    if df_tr[c].dtype == "object" or str(df_tr[c].dtype) == "category":
        cat_for_ohe.append(c)

df_ohe = pd.get_dummies(df_tr, columns=cat_for_ohe, drop_first=True)

# 8) Separar X, y y guardar archivos
if target_col and target_col in df_ohe.columns:
    y = df_ohe[target_col].astype(int) if df_ohe[target_col].dtype != int else df_ohe[target_col]
    X = df_ohe.drop(columns=[target_col] + id_cols, errors="ignore")
else:
    y = None
    X = df_ohe.drop(columns=id_cols, errors="ignore")

print("✅ Transformación completada.")
print("Shape X:", X.shape, "| y:", (y.shape if y is not None else None))

# 9) Exportar
X.to_csv("TelecomX_X_transformed.csv", index=False)
if y is not None:
    y.to_csv("TelecomX_y_target.csv", index=False)
df_ohe.to_csv("TelecomX_full_transformed.csv", index=False)
print("💾 Guardados: TelecomX_X_transformed.csv, TelecomX_y_target.csv (si aplica), TelecomX_full_transformed.csv")

# 10) Vista rápida
display(X.head())
if y is not None:
    print("\nDistribución de y (Churn):")
    print(y.value_counts(dropna=False))


✅ Transformación completada.
Shape X: (7267, 6560) | y: (7267,)
💾 Guardados: TelecomX_X_transformed.csv, TelecomX_y_target.csv (si aplica), TelecomX_full_transformed.csv


,customer.SeniorCitizen,customer.tenure,account.Charges.Monthly,total_services,customer.gender_Male,customer.Partner_Yes,customer.Dependents_Yes,phone.PhoneService_Yes,phone.MultipleLines_No phone service,phone.MultipleLines_Yes,...,account.Charges.Total_995.35,account.Charges.Total_996.45,account.Charges.Total_996.85,account.Charges.Total_996.95,account.Charges.Total_997.65,account.Charges.Total_997.75,account.Charges.Total_998.1,account.Charges.Total_999.45,account.Charges.Total_999.8,account.Charges.Total_999.9
0,0,9,65.6,0,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,9,59.9,0,True,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
2,0,4,73.9,0,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,13,98.0,0,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,3,83.9,0,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False



Distribución de y (Churn):
Churn
0    5398
1    1869
Name: count, dtype: int64


#📊 Carga y análisis

In [24]:
# ===========================================================
# TELECOMX - CARGA + ANÁLISIS (Colab ready)
# ===========================================================
import pandas as pd, numpy as np, json, io, re
from pathlib import Path

# ========= 1) CONFIGURA LA FUENTE =========
SOURCE_MODE = "github"   # "github" o "drive"

# A) GitHub RAW (ajusta si tu archivo está en otra URL)
GITHUB_RAW_URL = "https://raw.githubusercontent.com/fjvalencia7/challenge2-data-science-LATAM/main/TelecomX_Data.json"


# ========= 2) LECTORES ROBUSTOS =========
def _read_json_like_text(txt: str) -> pd.DataFrame:
    txt = txt.strip()
    # JSON Lines
    if "\n" in txt and txt.lstrip().startswith("{"):
        return pd.read_json(io.StringIO(txt), lines=True)
    # JSON lista/dict
    obj = json.loads(txt)
    return pd.json_normalize(obj)

def read_from_url(url: str) -> pd.DataFrame:
    import requests
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    u = url.lower()
    if u.endswith(".csv"):
        return pd.read_csv(io.StringIO(r.text))
    if u.endswith((".xlsx", ".xls")):
        return pd.read_excel(io.BytesIO(r.content))
    if u.endswith(".parquet"):
        return pd.read_parquet(io.BytesIO(r.content))
    # JSON por contenido
    return _read_json_like_text(r.text)

def read_from_path(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"No existe el archivo: {path}")
    low = str(p).lower()
    if low.endswith(".csv"):
        return pd.read_csv(p)
    if low.endswith((".xlsx", ".xls")):
        return pd.read_excel(p)
    if low.endswith(".parquet"):
        return pd.read_parquet(p)
    # JSON / JSONL
    try:
        return pd.read_json(p, lines=True)
    except Exception:
        with open(p, "r", encoding="utf-8") as f:
            txt = f.read()
        return _read_json_like_text(txt)

# ========= 3) EXTRACCIÓN =========
if SOURCE_MODE == "github":
    df = read_from_url(GITHUB_RAW_URL)
elif SOURCE_MODE == "drive":
    from google.colab import drive
    drive.mount('/content/drive')
    df = read_from_path(DRIVE_PATH)
else:
    raise ValueError("SOURCE_MODE debe ser 'github' o 'drive'.")

print("✅ Cargado. Dimensiones:", df.shape)
display(df.head())

# ========= 4) LIMPIEZA BÁSICA DE TIPOS =========
def to_bool_series(s):
    mapping = {
        "yes": True, "no": False, "si": True, "sí": True,
        "true": True, "false": False, "y": True, "n": False,
        "1": True, "0": False
    }
    return s.astype(str).str.strip().str.lower().map(mapping)

# Numéricos típicos
for col in ["tenure", "MonthlyCharges", "TotalCharges"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Booleanos típicos telco
for col in ["Partner","Dependents","PhoneService","PaperlessBilling",
            "MultipleLines","OnlineSecurity","OnlineBackup",
            "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Churn"]:
    if col in df.columns:
        df[col] = to_bool_series(df[col])

# Contract como ordinal si existe
if "Contract" in df.columns:
    order = ["Month-to-month","One year","Two year"]
    df["Contract"] = pd.Categorical(df["Contract"], categories=order, ordered=True)

print("\nTipos ajustados:")
print(df.dtypes)

# ========= 5) ANÁLISIS (EDA) =========
# 5.1 Diccionario de datos
def data_dictionary(dataframe):
    rows = []
    for c in dataframe.columns:
        s = dataframe[c]
        rows.append({
            "columna": c,
            "tipo": str(s.dtype),
            "nulos": int(s.isna().sum()),
            "cardinalidad": int(s.nunique()),
            "ejemplos": [str(v) for v in s.dropna().unique()[:5]]
        })
    return pd.DataFrame(rows).sort_values("columna")

print("\n=== DICCIONARIO DE DATOS ===")
dicc = data_dictionary(df)
display(dicc)

# 5.2 Nulos
print("\n=== NULOS (TOP 10) ===")
display(df.isna().sum().sort_values(ascending=False).head(10))

# 5.3 KPIs y tasa de churn
kpi = {}
kpi["n_clientes"] = len(df)
if "Churn" in df.columns:
    churn_rate = df["Churn"].mean() * 100
    kpi["churn_rate_%"] = round(churn_rate, 2)
if "MonthlyCharges" in df.columns:
    kpi["cargo_mensual_prom"] = round(df["MonthlyCharges"].mean(), 2)
if "tenure" in df.columns:
    kpi["tenure_medio_meses"] = round(df["tenure"].mean(), 2)

print("\n=== KPIs CLAVE ===")
display(pd.DataFrame([kpi]))

# 5.4 Cortes de churn por dimensiones clave
def rate_table(df, col, target="Churn"):
    if col not in df.columns or target not in df.columns:
        return pd.DataFrame()
    tmp = (df.groupby(col)[target]
             .mean()
             .mul(100).round(2)
             .rename("churn_%")
             .to_frame())
    tmp["n_clientes"] = df.groupby(col)[target].count()
    return tmp.sort_values("churn_%", ascending=False)

tablas = {}
for c in ["Contract","InternetService","PaymentMethod","SeniorCitizen","Partner","Dependents","PaperlessBilling"]:
    if c in df.columns and "Churn" in df.columns:
        tablas[f"churn_por_{c}"] = rate_table(df, c)

print("\n=== CHURN POR VARIABLES CLAVE ===")
for nombre, tabla in tablas.items():
    if not tabla.empty:
        print(f"\n-- {nombre} --")
        display(tabla)

# 5.5 Resumen numérico + correlaciones
num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
print("\n=== DESCRIPTIVO NUMÉRICO ===")
if num_cols:
    display(df[num_cols].describe().T)

# Correlación con churn (si existe y es binaria True/False)
if "Churn" in df.columns:
    # Convertir churn a 0/1 para correlación de Pearson (aprox. point-biserial)
    churn01 = df["Churn"].astype(float)
    corr = {}
    for c in num_cols:
        if c == "Churn":
            continue
        s = df[c]
        if s.notna().sum() > 2:
            corr[c] = churn01.corr(s)
    corr_df = (pd.Series(corr, name="corr_con_churn")
                 .sort_values(ascending=False)
                 .to_frame())
    print("\n=== CORRELACIÓN (numéricas) CON CHURN (≈ point-biserial) ===")
    display(corr_df.head(10))
    display(corr_df.tail(10))

# 5.6 Tenure buckets y churn por bucket
if "tenure" in df.columns and "Churn" in df.columns:
    buckets = pd.cut(df["tenure"],
                     bins=[-1, 6, 12, 24, 36, 60, np.inf],
                     labels=["0-6m","7-12m","13-24m","25-36m","37-60m","60m+"])
    churn_bucket = df.groupby(buckets)["Churn"].mean().mul(100).round(2).rename("churn_%")
    n_bucket = df.groupby(buckets)["Churn"].count().rename("n_clientes")
    churn_tenure = pd.concat([churn_bucket, n_bucket], axis=1).sort_index()
    print("\n=== CHURN POR TENURE (bucket) ===")
    display(churn_tenure)

# 5.7 Número de servicios activos y churn
svc_flags = [c for c in ["PhoneService","MultipleLines","OnlineSecurity","OnlineBackup",
                         "DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
             if c in df.columns]
if svc_flags and "Churn" in df.columns:
    # convertir a 0/1 si no lo están
    def to01(s):
        mapping = {"yes":1,"si":1,"sí":1,"true":1,"y":1,"1":1,
                   "no":0,"false":0,"n":0,"0":0}
        return (s.astype(str).str.strip().str.lower().map(mapping)).astype("float").fillna(0)
    df_svc = df.copy()
    for c in svc_flags:
        if df_svc[c].dtype == bool:
            df_svc[c] = df_svc[c].astype(int)
        elif not np.issubdtype(df_svc[c].dtype, np.number):
            df_svc[c] = to01(df_svc[c]).astype(int)
    df_svc["total_services"] = df_svc[svc_flags].sum(axis=1)
    churn_by_services = (df_svc.groupby("total_services")["Churn"]
                            .mean().mul(100).round(2)
                            .rename("churn_%").to_frame())
    churn_by_services["n_clientes"] = df_svc.groupby("total_services")["Churn"].count()
    print("\n=== CHURN POR NÚMERO DE SERVICIOS ===")
    display(churn_by_services)

# ========= 6) EXPORTAR RESULTADOS CLAVE =========
dicc.to_csv("TELX_diccionario.csv", index=False)
pd.DataFrame([kpi]).to_csv("TELX_kpis.csv", index=False)
for nombre, tabla in tablas.items():
    if not tabla.empty:
        tabla.to_csv(f"TELX_{nombre}.csv")
if num_cols:
    if "Churn" in df.columns:
        corr_df.to_csv("TELX_correlacion_con_churn.csv")
print("\n💾 Exportados CSVs de diccionario, KPIs y cortes de churn (si aplican).")


✅ Cargado. Dimensiones: (7267, 21)


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4



Tipos ajustados:
customerID                    object
Churn                         object
customer.gender               object
customer.SeniorCitizen         int64
customer.Partner              object
customer.Dependents           object
customer.tenure                int64
phone.PhoneService            object
phone.MultipleLines           object
internet.InternetService      object
internet.OnlineSecurity       object
internet.OnlineBackup         object
internet.DeviceProtection     object
internet.TechSupport          object
internet.StreamingTV          object
internet.StreamingMovies      object
account.Contract              object
account.PaperlessBilling      object
account.PaymentMethod         object
account.Charges.Monthly      float64
account.Charges.Total         object
dtype: object

=== DICCIONARIO DE DATOS ===


,columna,tipo,nulos,cardinalidad,ejemplos
1,Churn,object,224,2,"[False, True]"
19,account.Charges.Monthly,float64,0,1585,"[65.6, 59.9, 73.9, 98.0, 83.9]"
20,account.Charges.Total,object,0,6531,"[593.3, 542.4, 280.85, 1237.85, 267.4]"
16,account.Contract,object,0,3,"[One year, Month-to-month, Two year]"
17,account.PaperlessBilling,object,0,2,"[Yes, No]"
18,account.PaymentMethod,object,0,4,"[Mailed check, Electronic check, Credit card (..."
5,customer.Dependents,object,0,2,"[Yes, No]"
4,customer.Partner,object,0,2,"[Yes, No]"
3,customer.SeniorCitizen,int64,0,2,"[0, 1]"
2,customer.gender,object,0,2,"[Female, Male]"



=== NULOS (TOP 10) ===


...

,count,mean,std,min,25%,50%,75%,max
customer.SeniorCitizen,7267.0,0.162653,0.369074,0.00,0.000,0.0,0.000,1.00
customer.tenure,7267.0,32.346498,24.571773,0.00,9.000,29.0,55.000,72.00
account.Charges.Monthly,7267.0,64.720098,30.129572,18.25,35.425,70.3,89.875,118.75



=== CORRELACIÓN (numéricas) CON CHURN (≈ point-biserial) ===


,corr_con_churn
account.Charges.Monthly,0.193356
customer.SeniorCitizen,0.150889
customer.tenure,-0.352229


,corr_con_churn
account.Charges.Monthly,0.193356
customer.SeniorCitizen,0.150889
customer.tenure,-0.352229



💾 Exportados CSVs de diccionario, KPIs y cortes de churn (si aplican).


#📄Informe final

In [25]:
# ===========================================================
# INFORME FINAL - TelecomX (Colab Ready)
# Genera Markdown + HTML + Gráficos (PNG)
# ===========================================================
import pandas as pd, numpy as np, io, json, base64, textwrap, os
from pathlib import Path
import matplotlib.pyplot as plt

# ============== 0) CARGA (opcional) ========================
LOAD_FROM_GITHUB = True  # pon False si ya tienes df en memoria

if LOAD_FROM_GITHUB and 'df' not in globals():
    import requests
    GITHUB_RAW_URL = "https://raw.githubusercontent.com/fjvalencia7/challenge2-data-science-LATAM/main/TelecomX_Data.json"
    r = requests.get(GITHUB_RAW_URL, timeout=60); r.raise_for_status()
    txt = r.text.strip()
    if "\n" in txt and txt.lstrip().startswith("{"):
        df = pd.read_json(io.StringIO(txt), lines=True)
    else:
        obj = json.loads(txt); df = pd.json_normalize(obj)

# ============== 1) LIMPIEZA BÁSICA =========================
def to_bool_series(s):
    mapping = {"yes":True,"no":False,"si":True,"sí":True,"true":True,"false":False,"y":True,"n":False,"1":True,"0":False}
    return s.astype(str).str.strip().str.lower().map(mapping)

for col in ["tenure","MonthlyCharges","TotalCharges"]:
    if col in df.columns: df[col] = pd.to_numeric(df[col], errors="coerce")

for col in ["Partner","Dependents","PhoneService","PaperlessBilling",
            "MultipleLines","OnlineSecurity","OnlineBackup",
            "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Churn"]:
    if col in df.columns: df[col] = to_bool_series(df[col])

if "Contract" in df.columns:
    order = ["Month-to-month","One year","Two year"]
    df["Contract"] = pd.Categorical(df["Contract"], categories=order, ordered=True)

# ============== 2) MÉTRICAS Y TABLAS =======================
kpi = {"n_clientes": len(df)}
if "Churn" in df.columns and df["Churn"].notna().any():
    kpi["churn_rate_%"] = round(df["Churn"].mean()*100, 2)
if "MonthlyCharges" in df.columns:
    kpi["cargo_mensual_prom"] = round(df["MonthlyCharges"].mean(), 2)
if "tenure" in df.columns:
    kpi["tenure_medio_meses"] = round(df["tenure"].mean(), 2)

def rate_table(df, col, target="Churn"):
    if col not in df.columns or target not in df.columns: return pd.DataFrame()
    t = (df.groupby(col)[target].mean().mul(100).round(2).rename("churn_%").to_frame())
    t["n_clientes"] = df.groupby(col)[target].count()
    return t.sort_values("churn_%", ascending=False)

tab_contract  = rate_table(df, "Contract")          if "Contract" in df.columns else pd.DataFrame()
tab_internet  = rate_table(df, "InternetService")   if "InternetService" in df.columns else pd.DataFrame()
tab_paymethod = rate_table(df, "PaymentMethod")     if "PaymentMethod" in df.columns else pd.DataFrame()

num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
desc_num = df[num_cols].describe().T if num_cols else pd.DataFrame()

corr_df = pd.DataFrame()
if "Churn" in df.columns:
    churn01 = df["Churn"].astype(float)
    corr = {}
    for c in num_cols:
        if c == "Churn": continue
        s = df[c]
        if s.notna().sum() > 2:
            corr[c] = churn01.corr(s)
    if corr:
        corr_df = pd.Series(corr, name="corr_con_churn").sort_values(ascending=False).to_frame()

# Tenure buckets y churn
churn_tenure = pd.DataFrame()
if "tenure" in df.columns and "Churn" in df.columns:
    buckets = pd.cut(df["tenure"], bins=[-1,6,12,24,36,60,np.inf],
                     labels=["0-6m","7-12m","13-24m","25-36m","37-60m","60m+"])
    churn_bucket = df.groupby(buckets)["Churn"].mean().mul(100).round(2).rename("churn_%")
    n_bucket     = df.groupby(buckets)["Churn"].count().rename("n_clientes")
    churn_tenure = pd.concat([churn_bucket, n_bucket], axis=1).reset_index().rename(columns={"tenure":"bucket"})

# Número de servicios y churn
svc_flags = [c for c in ["PhoneService","MultipleLines","OnlineSecurity","OnlineBackup",
                         "DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
             if c in df.columns]
churn_by_services = pd.DataFrame()
if svc_flags and "Churn" in df.columns:
    df_svc = df.copy()
    for c in svc_flags:
        if df_svc[c].dtype == bool:
            df_svc[c] = df_svc[c].astype(int)
        elif not np.issubdtype(df_svc[c].dtype, np.number):
            df_svc[c] = to_bool_series(df_svc[c]).astype(float).fillna(0)
    df_svc["total_services"] = df_svc[svc_flags].sum(axis=1)
    churn_by_services = (df_svc.groupby("total_services")["Churn"].mean().mul(100).round(2)
                         .rename("churn_%").to_frame())
    churn_by_services["n_clientes"] = df_svc.groupby("total_services")["Churn"].count()
    churn_by_services = churn_by_services.reset_index()

# ============== 3) GRÁFICOS (PNG) ==========================
OUT_DIR = Path("informe_assets"); OUT_DIR.mkdir(exist_ok=True)
fig_paths = []

def save_bar(df_plot, x, y, title, fname):
    if df_plot.empty: return None
    plt.figure(figsize=(8,5))
    plt.bar(df_plot[x].astype(str), df_plot[y])
    plt.title(title)
    plt.xlabel(x); plt.ylabel(y)
    plt.xticks(rotation=45, ha='right')
    path = OUT_DIR/fname
    plt.tight_layout(); plt.savefig(path, dpi=120); plt.close()
    return str(path)

p1 = save_bar(tab_contract.reset_index().rename(columns={"index":"Contract"}), "Contract", "churn_%",
              "Churn por Contract", "churn_por_contract.png") if not tab_contract.empty else None
p2 = save_bar(tab_internet.reset_index().rename(columns={"index":"InternetService"}), "InternetService", "churn_%",
              "Churn por InternetService", "churn_por_internet.png") if not tab_internet.empty else None
p3 = save_bar(tab_paymethod.reset_index().rename(columns={"index":"PaymentMethod"}), "PaymentMethod", "churn_%",
              "Churn por PaymentMethod", "churn_por_payment.png") if not tab_paymethod.empty else None
p4 = save_bar(churn_tenure, "tenure", "churn_%",
              "Churn por Tenure (bucket)", "churn_por_tenure.png") if not churn_tenure.empty else None
p5 = save_bar(churn_by_services, "total_services", "churn_%",
              "Churn por Nº de servicios", "churn_por_servicios.png") if not churn_by_services.empty else None

fig_paths = [p for p in [p1,p2,p3,p4,p5] if p]

# ============== 4) CONSTRUIR INFORME =======================
def df_to_md_table(d, max_rows=12):
    if d is None or d.empty: return "_(sin datos)_"
    d2 = d.copy()
    if isinstance(d2, pd.Series): d2 = d2.to_frame()
    if len(d2) > max_rows: d2 = d2.head(max_rows)
    return d2.to_markdown(index=True)

def img_md(path):
    return f"![{Path(path).stem}]({path})" if path else ""

kpi_lines = []
for k,v in kpi.items():
    kpi_lines.append(f"- **{k.replace('_',' ').title()}**: {v}")
kpi_md = "\n".join(kpi_lines) if kpi_lines else "_(sin KPIs)_"

md = f"""
# Informe Final – TelecomX

**Objetivo:** Analizar métricas clave de clientes y contratos para entender la retención (churn) y priorizar acciones.

## 1) KPIs del dataset
{kpi_md}

## 2) Churn por dimensiones clave
### 2.1 Contract
{df_to_md_table(tab_contract)}

{img_md(p1)}

### 2.2 InternetService
{df_to_md_table(tab_internet)}

{img_md(p2)}

### 2.3 PaymentMethod
{df_to_md_table(tab_paymethod)}

{img_md(p3)}

## 3) Tenure y servicios
### 3.1 Churn por Tenure (bucket)
{df_to_md_table(churn_tenure)}

{img_md(p4)}

### 3.2 Churn por número de servicios
{df_to_md_table(churn_by_services)}

{img_md(p5)}

## 4) Resumen numérico y correlación
### 4.1 Descriptivo de variables numéricas
{df_to_md_table(desc_num)}

### 4.2 Correlación (numéricas) con churn (≈ point-biserial)
{df_to_md_table(corr_df)}

---

## 5) Hallazgos (resumen ejecutivo)
- La tasa de churn global se sitúa en **{kpi.get('churn_rate_%','N/A')}%**.
- **Contract** mensual suele asociarse a mayor churn que contratos de 1–2 años (ver tabla/figura).
- Los clientes con **menor tenure** presentan mayor probabilidad de churn.
- El número de **servicios activos** tiene relación con la permanencia: más servicios → menor churn (en general).
- Recomendaciones:
  1. Incentivar **upgrades de contrato** (mensual → 1 año) con beneficios.
  2. Paquetizar **servicios adicionales** (seguridad/backup/tech support) para aumentar “stickiness”.
  3. Revisar **métodos de pago** con mayor churn y ofrecer alternativas con descuentos.
"""

# Guardar MD
with open("Informe_TelecomX.md", "w", encoding="utf-8") as f:
    f.write(md)

# Convertir a HTML simple (Markdown básico)
try:
    import markdown
    html_body = markdown.markdown(md, extensions=["tables"])
except Exception:
    # Fallback muy simple (sin librería): envolver el md tal cual
    html_body = "<pre>" + md.replace("&","&amp;").replace("<","&lt;").replace(">","&gt;") + "</pre>"

html = f"""<!doctype html>
<html><head><meta charset="utf-8">
<title>Informe TelecomX</title>
<style>
body {{ font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica', Arial, sans-serif;
        max-width: 920px; margin: 24px auto; line-height: 1.5; }}
table {{ border-collapse: collapse; width: 100%; margin: 12px 0; }}
th, td {{ border: 1px solid #ddd; padding: 6px 8px; }}
th {{ background: #f4f4f4; text-align: left; }}
img {{ max-width: 100%; height: auto; margin: 8px 0; }}
code, pre {{ background:#f6f8fa; padding: 2px 4px; border-radius:4px; }}
</style></head><body>
{html_body}
</body></html>"""

with open("Informe_TelecomX.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ Informe generado:")
print("- Informe_TelecomX.md")
print("- Informe_TelecomX.html")
print("- Carpeta de imágenes:", OUT_DIR)


✅ Informe generado:
- Informe_TelecomX.md
- Informe_TelecomX.html
- Carpeta de imágenes: informe_assets
